In [1]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Path to the directory containing the audio files
data_dir = f"{PATH}/genres_original"

# Get the list of audio files
audio_files = []
labels = []
features = []
for genre in os.listdir(data_dir):
    genre_dir = os.path.join(data_dir, genre)
    if os.path.isdir(genre_dir):
        for file in os.listdir(genre_dir):
            if file.endswith(".wav") and not file.endswith("jazz.00054.wav"):
                try:
                    audio_files.append(os.path.join(genre_dir, file))
                    labels.append(genre)
                except:
                    print("Error encountered while parsing file: ", file)

# Load and extract features from the audio files

for file in audio_files:
    try:
        audio, sr = librosa.load(file, sr=None, mono=True, res_type='kaiser_best', duration=None, offset=0.0, dtype=np.float32)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr)
        features.append(mfcc)
    except:
        print("Error encountered while parsing file: ", file)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-05-29 01:15:52.552923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("Number of files processed: ", len(features))

Number of files processed:  999


In [3]:
# Encode the labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [9]:
from keras.utils import pad_sequences

# Convert features and labels to numpy arrays
features_padded =[]
for feature in features:
    features_padded.append(pad_sequences(feature, padding='post', maxlen=1400))

features = np.array(features_padded)
labels_encoded = np.array(labels_encoded)

# Check the shapes of the arrays
print("Shape of features:", features.shape)
print("Shape of labels_encoded:", labels_encoded.shape)

# # Remove the last sample from labels_encoded
# labels_encoded = labels_encoded[:-1]

# Check the updated shape of labels_encoded
print("Updated shape of labels_encoded:", labels_encoded.shape)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)


Shape of features: (999, 20, 1400)
Shape of labels_encoded: (999,)
Updated shape of labels_encoded: (999,)


In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

# ... Load and preprocess your data ...

# Flatten the input data
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Convert the target data to categorical target data
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)


# Check the shape of the flattened data
print(X_train_flat.shape, y_train_encoded.shape)

# Define the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_flat.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))



# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_flat, y_train_encoded, epochs=500, batch_size=32, validation_data=(X_test_flat, y_test_encoded))

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_flat, y_test_encoded)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


(799, 28000) (799, 10)
Epoch 1/500
25/25 [==============================] - 1s 14ms/step - loss: 125.9506 - accuracy: 0.2003 - val_loss: 60.4096 - val_accuracy: 0.3000
Epoch 2/500
25/25 [==============================] - 0s 8ms/step - loss: 37.0596 - accuracy: 0.4155 - val_loss: 35.5463 - val_accuracy: 0.4300
Epoch 3/500
25/25 [==============================] - 0s 8ms/step - loss: 15.1340 - accuracy: 0.6233 - val_loss: 49.4558 - val_accuracy: 0.3750
Epoch 4/500
25/25 [==============================] - 0s 8ms/step - loss: 12.3115 - accuracy: 0.6521 - val_loss: 31.7029 - val_accuracy: 0.3500
Epoch 5/500
25/25 [==============================] - 0s 10ms/step - loss: 8.2666 - accuracy: 0.7409 - val_loss: 37.4853 - val_accuracy: 0.3200
Epoch 6/500
25/25 [==============================] - 0s 9ms/step - loss: 5.0628 - accuracy: 0.7972 - val_loss: 34.1410 - val_accuracy: 0.4000
Epoch 7/500
25/25 [==============================] - 0s 9ms/step - loss: 3.1417 - accuracy: 0.8673 - val_loss: 36.4809

In [18]:
def classify_mov_file(file_path, model, label_encoder):
    # Step 1: Load the .wav file and extract the audio
    audio, sr = librosa.load(file_path, sr=None, mono=True, res_type='kaiser_best', duration=None, offset=0.0, dtype=np.float32)

    # Step 2: Preprocess the audio data to obtain features
    mfcc = librosa.feature.mfcc(y=audio, sr=sr)
    features = pad_sequences(mfcc, padding='post', maxlen=1400)  # Pad the sequence with zeros

    # Step 3: Reshape the features to match the input shape of the model
    features_flattened = features.reshape(1, -1)

    # Step 4: Use the trained model to predict the class
    predicted_probabilities = model.predict(features_flattened)
    predicted_class = np.argmax(predicted_probabilities)

    # Step 5: Convert the predicted class index to a genre string
    predicted_genre = label_encoder.inverse_transform([predicted_class])

    return predicted_genre[0]



In [33]:
classify_mov_file(FILE,model, label_encoder)

1/1 [==============================] - 0s 19ms/step


'classical'